In [1]:
import os
import pprint
import pymongo

from xml.etree import ElementTree as ET
from datetime import datetime
from dotenv import dotenv_values
from SPARQLWrapper import SPARQLWrapper, POST, DIGEST, JSON

In [2]:
env_config = dotenv_values("../environment/.env")

mongo_client = pymongo.MongoClient(
    host=env_config['MONGODB_HOST'],
    port=int(env_config['MONGODB_PORT']),
    username=env_config['MONGO_USER'],
    password=env_config['MONGO_PASSWORD'],
)
mongo_collection = mongo_client["boe_db"]["boe"]

In [3]:
# Set up the SPARQL connection to GraphDB
sparql = SPARQLWrapper("http://localhost:7200/repositories/your_repository/statements")
sparql.setHTTPAuth(DIGEST)
sparql.setMethod(POST)

# Define your SPARQL query for data insertion
sparql.setQuery("""
    PREFIX ex: <http://example.com/ontology#>
    INSERT DATA {
        ex:individual1 a ex:YourClass ;
            ex:hasProperty "Some Value" .
    }
""")

# Execute the query
sparql.query()


EndPointNotFound: EndPointNotFound: It was not possible to connect to the given endpoint: check it is correct. 

Response:
b'Unknown repository: your_repository'

## Investigando las relaciones "anterior" y "posterior"

In [13]:
anteriores = dict()
## get all from collection with a "materias" field of more than 1 element
for item in mongo_collection.find({"anteriores": {"$exists": True, "$not": {"$size": 0}}}):
    for anterior in item["anteriores"]:
        relacion = anterior["relacion"]
        anteriores[relacion["codigo"]] = relacion["texto"]
pprint.pprint(anteriores)

{'201': 'CORRECCIÓN de errores',
 '202': 'CORRECCIÓN de erratas',
 '203': 'CORRIGE errores',
 '210': 'DEROGA',
 '230': 'DEJA SIN EFECTO',
 '231': 'SUSPENDE',
 '247': 'ENMIENDAS',
 '270': 'MODIFICA',
 '300': 'PUBLICA',
 '330': 'CITA',
 '331': 'EN RELACIÓN con',
 '401': 'PRORROGA',
 '404': 'ACTUALIZA',
 '406': 'AMPLÍA',
 '407': 'AÑADE',
 '420': 'APRUEBA',
 '426': 'TRANSPONE',
 '440': 'DE CONFORMIDAD con',
 '470': 'DECLARA',
 '490': 'DESARROLLA',
 '552': 'Recurso promovido contra',
 '693': 'DICTADA'}


In [14]:
posteriores = dict()
## get all from collection with a "materias" field of more than 1 element
for item in mongo_collection.find({"posteriores": {"$exists": True, "$not": {"$size": 0}}}):
    for anterior in item["posteriores"]:
        relacion = anterior["relacion"]
        posteriores[relacion["codigo"]] = relacion["texto"]
pprint.pprint(posteriores)

{'201': 'CORRECCIÓN de errores',
 '202': 'CORRECCIÓN de erratas',
 '203': 'SE CORRIGEN errores',
 '230': 'SE DEJA SIN EFECTO',
 '270': 'SE MODIFICA',
 '300': 'SE PUBLICA',
 '301': 'SE PUBLICA  Acuerdo de convalidación',
 '406': 'SE AMPLÍA',
 '420': 'SE APRUEBA'}
